## 🍀 Operations Research Application HW1
- Author: 楊晴雯
- Date: 2022/09/10
- NCKU-ORA: Assignment 1 (Code)

## Q1

In [1]:
from gurobipy import *
import pandas as pd

In [2]:
data = {'FatCalories': [10,75,0,20,70,0],
    'TotalCalories' : [70,100,50,60,150,100],
    'VitaminC' : [0,0,3,0,2,120],
    'Protein' : [3,4,0,1,8,1],
    'Cost' : [5,4,7,8,15,35]}
data = pd.DataFrame(data)

In [3]:
data

,FatCalories,TotalCalories,VitaminC,Protein,Cost
0,10,70,0,3,5
1,75,100,0,4,4
2,0,50,3,0,7
3,20,60,0,1,8
4,70,150,2,8,15
5,0,100,120,1,35


In [4]:
model = Model('Q1')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-31


In [11]:
# number of food items 
N = 6 
Mapping = ['bread', 'peanut-butter', 'jelly', 'cracker', 'milk', 'juice']
def modeling(vartype):
    X = [0 for _ in range(N)]
    for i in range(N):
        if vartype == 'continuous':
            X[i] = model.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = f'x{i+1}') #sign + constraint
        else: 
            X[i] = model.addVar(lb = 0, vtype = GRB.INTEGER, name = f'x{i+1}') #sign + constraint
        
        
    # Objective: cost minimization    
    total_cost = quicksum(data.Cost[i] * X[i] for i in range(N))
    model.setObjective(total_cost, GRB.MINIMIZE)

    # 1. Total Calories Constraint 
    Ct = quicksum(data.TotalCalories[i] * X[i] for i in range(N))
    model.addConstr(Ct <= 600, 'total calories upper')
    model.addConstr(Ct >= 400, 'total calories lower')


    # 2. Calories Fat Percentage 
    Cf = quicksum(data.FatCalories[i] * X[i] for i in range(N))
    model.addConstr(Cf <= Ct*0.3, "calories fat percentage")

    # 3. Vitamin C intake 
    VCI = quicksum(data.VitaminC[i] * X[i] for i in range(N))
    model.addConstr(VCI >= 60, 'Vitamin C intake')

    # 4. Protein
    PI = quicksum(data.Protein[i] * X[i] for i in range(N))
    model.addConstr(PI >= 12, 'Protein intake')

    # 5. Extra 
    model.addConstr(X[0] == 2, 'X1 (bread) should be exactly 2 pcs' )
    model.addConstr(X[1] >= 2*X[2], 'X2 (peanut butter) should be at least twice as much as X3 (jelly)')
    model.addConstr(X[4]+X[5] >= 1, 'at least 1 cup of liquid(X5, X6)')
    return model, X

In [17]:
model,X = modeling('continuous')
model.optimize()
min_cost = model.objVal
print(f'* minimum cost per child: {min_cost}')
for i in range(6):
    print(f'{Mapping[i]}: {X[i].x}')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 30 columns and 155 nonzeros
Model fingerprint: 0xac515d0c
Variable types: 18 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+02]

MIP start from previous solve produced solution with objective 47.3106 (0.01s)
Loaded MIP start from previous solve with objective 47.3106

Presolve removed 33 rows and 25 columns
Presolve time: 0.00s
Presolved: 7 rows, 5 columns, 26 nonzeros
Variable types: 5 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        47.31063

In [15]:
model,X = modeling('integer')
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 32 rows, 24 columns and 124 nonzeros
Model fingerprint: 0xea0d8e50
Variable types: 12 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+02]

MIP start from previous solve produced solution with objective 85 (0.01s)
MIP start from previous solve produced solution with objective 57 (0.01s)
Loaded MIP start from previous solve with objective 57

Presolve removed 26 rows and 19 columns
Presolve time: 0.01s
Presolved: 6 rows, 5 columns, 20 nonzeros
Variable types: 5 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 57 85 

Optimal solution found (tolerance 1.00e-04)
Best objec

In [16]:
min_cost = model.objVal
print(f'* minimum cost per child: {min_cost}')
for i in range(6):
    print(f'{Mapping[i]}: {X[i].x}')

* minimum cost per child: 57.0
bread: 2.0
peanut-butter: 1.0
jelly: 0.0
cracker: 1.0
milk: -0.0
juice: 1.0


## Q2

In [40]:
pip -q install discreteMarkovChain

Note: you may need to restart the kernel to use updated packages.


In [65]:
import numpy as np
from discreteMarkovChain import markovChain

# U: the number of components that are operational st the end of period t
# V: the number of periods of repair that have been completed on components that are not yet operational 

states = [(2,0), (1,0), (0,1), (1,1)] # (U, V)
markov_matrix = [[0.8,0.2,0,0],
                [0,0,0.2,0.8],
                [0,1,0,0],
                [0.8,0.2,0,0]]
P = np.matrix(markov_matrix)

# the probability of reaching state 2 from the {n}th transition matrix
# Let initial probs (M) be (U, V) = (2,0) as 1 (both components are operational)
# https://www.math.ncku.edu.tw/~fang/Linear-Algebra-Note-06.pdf
# state probibilities after n periods: P^(n-1)M, given that P: row: present state, col: next state

M = np.array([1, 0, 0, 0])
print('(2) The probability that the unit will be inoperable')
for n in [2,5,10,20]:
    Ppower = np.linalg.matrix_power(P, n-1)
    probs = np.dot(M, Ppower)
    print(f'\tafter {n} periods: {probs[:,2]}') 

mc = markovChain(P)
mc.computePi('linear') #We can also use 'power', 'krylov' or 'eigen'
print(f'(3) steady state: {mc.pi}')

cost = 30000
print(f'(4) The long-run expected cost is ${mc.pi[2]*cost}.')

(2) The probability that the unit will be inoperable
	after 2 periods: [[0.]]
	after 5 periods: [[0.04]]
	after 10 periods: [[0.03838157]]
	after 20 periods: [[0.03846144]]
(3) steady state: [0.61538462 0.19230769 0.03846154 0.15384615]
(4) The long-run expected cost is $1153.8461538461538.
